In [5]:
# Імпортувати необхідні бібліотеки
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import joblib

# === 1. Завантажити набір даних ===
data = pd.read_csv('IMDB Dataset.csv')

print("Колонки у файлі:", data.columns.tolist())

# === 2. Формування корпусу текстів і міток ===
# Об’єднуємо позитивні та негативні відгуки в один набір
pos = data['Позитивні відгуки'].fillna("").astype(str)
neg = data['Негативні відгуки'].fillna("").astype(str)

# Створюємо текстовий корпус (всі відгуки)
x = pd.concat([pos, neg], axis=0).reset_index(drop=True)

# Створюємо мітки: 1 — позитивний, 0 — негативний
y = np.array(['positive'] * len(pos) + ['negative'] * len(neg))

# Якщо потрібно обмежити до 2500 записів:
x = x[:2500]
y = y[:2500]

# === 3. Токенізація ===
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)
maxlen = 50
x = pad_sequences(sequences, maxlen=maxlen)

# === 4. Кодування міток ===
binr = OneHotEncoder(sparse_output=False)
y = binr.fit_transform(np.array(y).reshape(-1, 1))

# === 5. Побудова моделі ===
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=16))
model.add(Bidirectional(LSTM(16, return_sequences=True)))
model.add(LSTM(16, return_sequences=True))
model.add(LSTM(16))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

# === 6. Компіляція ===
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

# === 7. Навчання ===
history = model.fit(x, y, epochs=5, batch_size=64, validation_split=0.2)

# === 8. Збереження ===
model.save('sentiment_analysis_model.h5')
joblib.dump(tokenizer, 'tokenizer.joblib')
joblib.dump(binr, 'encoder.joblib')

print("\n✅ Модель успішно навчена і збережена!")

Колонки у файлі: ['автор', 'нація', 'рахунок', 'назва', 'Позитивні відгуки', 'Негативні відгуки', 'Дата коментаря', 'Тип кімнати', 'Дата заїзду', 'Дата виїзду']
Epoch 1/5
16/16 [==============================] - 11s 192ms/step - loss: 0.6794 - acc: 0.6240 - val_loss: 0.9567 - val_acc: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 1s 72ms/step - loss: 0.6582 - acc: 0.6250 - val_loss: 0.9920 - val_acc: 0.0000e+00
Epoch 3/5
16/16 [==============================] - 1s 71ms/step - loss: 0.6482 - acc: 0.6260 - val_loss: 1.1999 - val_acc: 0.0000e+00
Epoch 4/5
16/16 [==============================] - 1s 78ms/step - loss: 0.6232 - acc: 0.6732 - val_loss: 1.2839 - val_acc: 0.0000e+00
Epoch 5/5
16/16 [==============================] - 1s 73ms/step - loss: 0.5755 - acc: 0.7185 - val_loss: 0.7807 - val_acc: 0.0000e+00

✅ Модель успішно навчена і збережена!


In [6]:
# views.py
from django.shortcuts import render
from django.http import HttpResponse
import joblib

def sentiment_analysis_view(request):
    if request.method == 'POST':
        # Get the user input
        text = request.POST['text']

        # Load the saved NLP model
        model = joblib.load('sentiment_analysis_model.joblib')

        # Perform the prediction
        prediction = model.predict([text])[0]

        # Render the template with the prediction result
        return render(request, 'sentiment_analysis.html', {'prediction': prediction})

In [8]:
# urls.py
from django.urls import path
from .views import sentiment_analysis_view

urlpatterns = [
    path('sentiment/', sentiment_analysis_view, name='sentiment_analysis'),
    # Add other URL patterns as needed
]

ImportError: attempted relative import with no known parent package